In [ ]:
import sys
import os
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from typing import List


PROJECT_ROOT = os.path.abspath(os.path.join(os.path.abspath('../..'),os.pardir))
sys.path.append(PROJECT_ROOT)

from electronTransportCode.MCEstimator import TrackEndEstimator
%matplotlib widget

In [ ]:
particleTracerK = pickle.load(open('data/particleTracerK.pkl', 'rb'))
particleTracerKD = pickle.load(open('data/particleTracerKD.pkl', 'rb'))
eSource, NB_PARTICLES, dsArray = pickle.load(open('data/simargv.pkl', 'rb'))
nbSim = dsArray.size

In [ ]:
print(eSource, NB_PARTICLES, dsArray)

In [ ]:
# Load all simulation results from data/
TrackEndEstimatorK: TrackEndEstimator = pickle.load(open('data/TrackEndEstimatorK.pkl', 'rb'))[0]

TEKDList: List[TrackEndEstimator] = []
TEKDSList: List[TrackEndEstimator] = []
for i in range(nbSim):
    TEKDList.append(pickle.load(open(f'data/TrackEndEstimatorKD{i}.pkl', 'rb'))[0])
    TEKDSList.append(pickle.load(open(f'data/TrackEndEstimatorKDS{i}.pkl', 'rb'))[0])


In [ ]:
# Compute wasserstein distance
kCenter, kData = TrackEndEstimatorK.getDensityData()

Wass = np.empty((nbSim, 2), dtype=float)
for i in range(nbSim):
    _, kdData = TEKDList[i].getDensityData()
    _, kdsData = TEKDSList[i].getDensityData()
    Wass[i, 0] = stats.wasserstein_distance(kData, kdData)
    Wass[i, 1] = stats.wasserstein_distance(kData, kdsData)


In [ ]:
# Plotting
fig, ax1 = plt.subplots(1, 1, figsize=(15, 3.9))

ax1.plot(dsArray, Wass[i, 0], 'r', label='KDMC')
ax1.plot(dsArray, Wass[i, 1], '--b', label= 'KDSMC')
ax1.set_xlabel('$\Delta s$')
ax1.set_ylabel('Wasserstein distance')